# Diff-Diff analysis

In [1]:
import numpy as np
import pandas as pd
import altair as alt

In [2]:
df_treat = pd.read_csv('year_wise_treatment.csv')
df_control = pd.read_csv('year_wise_control.csv')

In [4]:
df_treat.year = df_treat.year.astype('int')
df_control.year = df_control.year.astype('int')

### Loess Fit (may need scale y)

Treated group

In [138]:

base = alt.Chart(df_treat, title = "Average cost per dosage by year(treated group)").mark_point(\
    ).encode(x = alt.X('year', title = 'Year', scale = alt.Scale(zero=False),\
    axis = alt.Axis(values=[2017, 2018, 2019, 2020, 2021], format = '.0f')),\
     y = alt.Y('Avg_Spnd_Per_Dsg_Unt_Wghtd', title = 'Average spending per dosage unit weight'))

fit = base.transform_loess('year', 'Avg_Spnd_Per_Dsg_Unt_Wghtd').mark_line(color = 'red')

base + fit

alt.LayerChart(...)

Control group

In [136]:
alt.data_transformers.disable_max_rows()

base = alt.Chart(df_control, title = "Average cost per dosage by year(control group)").mark_point(\
    ).encode(x = alt.X('year', title = 'Year', scale = alt.Scale(zero=False), \
        axis = alt.Axis(values=[2017, 2018, 2019, 2020, 2021], format = '.0f')),\
     y = alt.Y('Avg_Spnd_Per_Dsg_Unt_Wghtd', title = 'Average spending per dosage unit weight'))

fit = base.transform_loess('year', 'Avg_Spnd_Per_Dsg_Unt_Wghtd').mark_line(color = 'red')

base + fit

alt.LayerChart(...)


#### Loess fit with scaled price data ?

### Linear fit + CI interval

In [129]:
def get_reg_fit(data, yvar, xvar, alpha=0.05, color = 'blue'):
    """function to plot regression fit with CI interval"""
    import statsmodels.formula.api as smf

    # Grid for predicted values
    x = data.loc[pd.notnull(data[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions = pd.DataFrame({xvar: grid})

    # Fit model, get predictions
    model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
    model_predict = model.get_prediction(predictions[xvar])
    predictions[yvar] = model_predict.summary_frame()["mean"]
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)

    # Build chart
    reg = alt.Chart(predictions).mark_line(color = color).encode(x=\
        alt.X(xvar, title ='Year', axis = alt.Axis(values=[2017, 2018, 2019, 2020, 2021], format = '.0f')), 
        y= alt.Y(yvar, title ='Average drug price per dosage'))
    ci = (
        alt.Chart(predictions)
        .mark_errorband(color = color)
        .encode(
            x=alt.X(xvar, title ='Year', axis = alt.Axis(values=[2017, 2018, 2019, 2020, 2021], format = '.0f')), 
            y=alt.Y('ci_low', title ='Average drug price per dosage'),
            y2="ci_high",
        )
    )
    chart = ci + reg
    return predictions, chart


In [130]:
df_pre_treat = df_treat[df_treat['year'].isin([year for year in [2017, 2018, 2019]])]
df_post_treat = df_treat[df_treat['year'].isin([year for year in [2020, 2021]])]

df_pre_control = df_control[df_control['year'].isin([year for year in [2017, 2018, 2019]])]
df_post_control = df_control[df_control['year'].isin([year for year in [2020, 2021]])]

In [131]:
fit_pre1, reg_chart_pre1 = get_reg_fit(
   df_pre_treat , yvar="Avg_Spnd_Per_Dsg_Unt_Wghtd", xvar="year", alpha=0.05, color = 'red'
)
fit_post1, reg_chart_post1 = get_reg_fit(
   df_post_treat , yvar="Avg_Spnd_Per_Dsg_Unt_Wghtd", xvar="year", alpha=0.05, color = 'red'
)
fit_pre0, reg_chart_pre0 = get_reg_fit(
   df_pre_control , yvar="Avg_Spnd_Per_Dsg_Unt_Wghtd", xvar="year", alpha=0.05
)
fit_post0, reg_chart_post0 = get_reg_fit(
   df_post_control , yvar="Avg_Spnd_Per_Dsg_Unt_Wghtd", xvar="year", alpha=0.05
)
# add vertical line for the legalization year
vertline = (
        alt.Chart(pd.DataFrame({"year": 2019, "color": ["black"]}))
        .mark_rule(strokeDash=[5, 3])
        .encode(x="year:Q", color=alt.Color("color:N", scale=None))
    )

In [132]:
plot = alt.layer(reg_chart_pre1 + reg_chart_pre0 + reg_chart_post1 + reg_chart_post0 + vertline).properties(\
    title = 'Average drug price per dosage v.s. Year (Blue: control, Red: treated)')

plot

alt.LayerChart(...)

In [19]:
### code without CI interval
# df_pre_plot = df[df['year'].isin([year for year in [2017, 2018, 2019]])]
# df_post_plot = df[df['year'].isin([year for year in [2020, 2021]])]

# df_pre_plot1 = df_control[df_control['year'].isin([year for year in [2017, 2018, 2019]])]
# df_post_plot1 = df_control[df_control['year'].isin([year for year in [2020, 2021]])]

# # pre-expiration
# base_pre_c = alt.Chart(df_pre_plot, \
#     title = 'Prescription drug price over years').mark_point().encode(
#     x= alt.X('year', title = 'Year', scale = alt.Scale(zero=False)),\
#     y = alt.Y('Avg_Spnd_Per_Dsg_Unt_Wghtd', title = 'Spending per dosage', scale = alt.Scale(zero=False))
# )
# fit_pre_c = base_pre_c.transform_regression('year', 'Avg_Spnd_Per_Dsg_Unt_Wghtd').mark_line(color='red')

# # post-expiration
# base_post_c = alt.Chart(df_post_plot, \
#     title = 'Prescription drug price over years').mark_point().encode(
#     x= alt.X('year', title = 'Year', scale = alt.Scale(zero=False)),\
#     y = alt.Y('Avg_Spnd_Per_Dsg_Unt_Wghtd', title = 'Spending per dosage', scale = alt.Scale(zero=False))
# )
# fit_post_c = base_post_c.transform_regression('year', 'Avg_Spnd_Per_Dsg_Unt_Wghtd').mark_line(color='red')

# # pre-expiration control
# base_pre_c1 = alt.Chart(df_pre_plot1, \
#     title = 'Prescription drug price over years').mark_point().encode(
#     x= alt.X('year', title = 'Year', scale = alt.Scale(zero=False)),\
#     y = alt.Y('Avg_Spnd_Per_Dsg_Unt_Wghtd', title = 'Spending per dosage', scale = alt.Scale(zero=False))
# )
# fit_pre_c1 = base_pre_c1.transform_regression('year', 'Avg_Spnd_Per_Dsg_Unt_Wghtd').mark_line(color='blue')

# # post-expiration control
# base_post_c1 = alt.Chart(df_post_plot1, \
#     title = 'Prescription drug price over years').mark_point().encode(
#     x= alt.X('year', title = 'Year', scale = alt.Scale(zero=False)),\
#     y = alt.Y('Avg_Spnd_Per_Dsg_Unt_Wghtd', title = 'Spending per dosage', scale = alt.Scale(zero=False))
# )
# fit_post_c1 = base_post_c1.transform_regression('year', 'Avg_Spnd_Per_Dsg_Unt_Wghtd').mark_line(color='blue')

# # add vertical line for the legalization year
# vertline = (
#         alt.Chart(pd.DataFrame({"Date": 2019, "color": ["green"]}))
#         .mark_rule(strokeDash=[5, 3])
#         .encode(x="Date:Q", color=alt.Color("color:N", scale=None))
#     )

# fit_post_c + fit_pre_c + fit_pre_c1 + fit_post_c1 + vertline